## Deliverable 2. Create a Customer Travel Destinations Map.

In [2]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [3]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Bambous Virieux,MU,-20.34,57.76,70.00,94,40,2.24,scattered clouds
1,1,Butaritari,KI,3.07,172.79,81.30,75,100,14.25,overcast clouds
2,2,Teknaf,BD,20.86,92.31,70.29,77,0,4.09,clear sky
3,3,Punta Arenas,CL,-53.15,-70.92,51.80,53,75,12.75,broken clouds
4,4,Ambovombe,MG,-25.17,46.08,70.90,68,100,2.17,overcast clouds


In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Butaritari,KI,3.07,172.79,81.30,75,100,14.25,overcast clouds
5,5,Busselton,AU,-33.65,115.33,75.00,37,0,9.13,clear sky
6,6,Manggar,ID,-2.88,108.27,83.53,73,100,9.31,light rain
9,9,Puerto Escondido,MX,15.85,-97.07,82.40,61,75,5.50,broken clouds
10,10,Kapaa,US,22.08,-159.32,82.40,74,75,12.75,broken clouds
11,11,Porto Novo,BJ,6.50,2.60,82.40,83,40,12.75,scattered clouds
16,16,Vaini,TO,-21.20,-175.20,78.80,74,40,10.29,scattered clouds
20,20,Bintulu,MY,3.17,113.03,80.60,88,20,3.36,few clouds
21,21,Camacari,BR,-12.70,-38.32,78.80,88,40,10.29,scattered clouds
26,26,Matamoros,MX,25.53,-103.25,78.80,32,5,4.70,clear sky


In [6]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum

<bound method DataFrame.sum of      City_ID   City  Country    Lat    Lng  Max Temp  Humidity  Cloudiness  \
1      False  False    False  False  False     False     False       False   
5      False  False    False  False  False     False     False       False   
6      False  False    False  False  False     False     False       False   
9      False  False    False  False  False     False     False       False   
10     False  False    False  False  False     False     False       False   
..       ...    ...      ...    ...    ...       ...       ...         ...   
670    False  False    False  False  False     False     False       False   
673    False  False    False  False  False     False     False       False   
677    False  False    False  False  False     False     False       False   
679    False  False    False  False  False     False     False       False   
682    False  False    False  False  False     False     False       False   

     Wind Speed  Current Descrip

In [7]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_hotel_df = preferred_cities_df.dropna()

In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_hotel_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Butaritari,KI,81.30,overcast clouds,3.07,172.79,
5,Busselton,AU,75.00,clear sky,-33.65,115.33,
6,Manggar,ID,83.53,light rain,-2.88,108.27,
9,Puerto Escondido,MX,82.40,broken clouds,15.85,-97.07,
10,Kapaa,US,82.40,broken clouds,22.08,-159.32,
11,Porto Novo,BJ,82.40,scattered clouds,6.50,2.60,
16,Vaini,TO,78.80,scattered clouds,-21.20,-175.20,
20,Bintulu,MY,80.60,few clouds,3.17,113.03,
21,Camacari,BR,78.80,scattered clouds,-12.70,-38.32,
26,Matamoros,MX,78.80,clear sky,25.53,-103.25,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")  

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [10]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.dropna()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Butaritari,KI,81.30,overcast clouds,3.07,172.79,Isles Sunset Lodge
5,Busselton,AU,75.00,clear sky,-33.65,115.33,Observatory Guest House
6,Manggar,ID,83.53,light rain,-2.88,108.27,Guest Hotel
9,Puerto Escondido,MX,82.40,broken clouds,15.85,-97.07,Aldea del Bazar Hotel and Spa
10,Kapaa,US,82.40,broken clouds,22.08,-159.32,Sheraton Kauai Resort at Coconut Beach
...,...,...,...,...,...,...,...
670,Tabou,CI,77.59,overcast clouds,4.42,-7.35,hotêl doufoulougou
673,Mackay,AU,84.20,overcast clouds,-21.15,149.20,International Lodge Motel
677,Diamantino,BR,80.22,overcast clouds,-14.41,-56.45,Hotel Delta
679,Bachaquero,VE,81.03,scattered clouds,9.97,-71.15,casa de los dinosaurios


In [11]:
# 8a. Create the output File (CSV)
output_data_file = "Weather_Database/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [12]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [13]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))